# Jeke xG Model

Modelo de Expected Goals (xG) basado en **decaimiento exponencial** con distancia euclidiana.

## Estructura
1. **Descarga de datos** - Obtener tiros de Understat
2. **Preprocesamiento** - Limpieza y estandarización
3. **Distancia euclidiana** - Cálculo de distancia al centro de la portería
4. **Modelo exponencial** - Ajuste y visualización
5. **Análisis por equipo** - Rankings ofensivo y defensivo

In [ ]:
from understatapi import UnderstatClient
import pandas as pd
import numpy as np
from time import sleep
from tqdm.auto import tqdm
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [ ]:
# Configuración
LEAGUE = "EPL"    # "EPL", "La_Liga", "Bundesliga", "Serie_A", "Ligue_1", "RFPL"
SEASON = ["2025"]

# Dimensiones del campo (metros)
PITCH_LENGTH_M = 100.0
PITCH_WIDTH_M = 65.0
GOAL_CENTER_Y = 0.5

# Parámetros de binning y filtrado
BIN_WIDTH = 1.0
ALPHA = 1.0      # Suavizado de Laplace
LOW_N = 10       # Mínimo de tiros por bin
D_MIN = 2.0      # Distancia mínima para regresión
D_MAX = 33.0     # Distancia máxima para regresión

---
## 1. Descarga de datos

Obtenemos los datos de tiros desde la API de Understat.

In [ ]:
def get_league_match_ids(league: str, seasons: list[str]):
    """Retorna lista de match_ids y metadatos por liga y temporadas."""
    all_match_ids = []
    all_matches = []
    with UnderstatClient() as us:
        for season in seasons:
            matches = us.league(league=league).get_match_data(season=season)
            all_match_ids.extend([m["id"] for m in matches])
            all_matches.extend(matches)
    return all_match_ids, all_matches

def fetch_match_shots(match_id: str, client: UnderstatClient) -> list[dict]:
    """Obtiene tiros de un partido."""
    out = []
    md = client.match(match=match_id).get_shot_data()
    for side in ("h", "a"):
        for s in md.get(side, []):
            s = dict(s)
            s["h_a"] = side
            s["match_id"] = match_id
            out.append(s)
    return out

def shots_to_df(shots: list[dict]) -> pd.DataFrame:
    """Convierte lista de tiros a DataFrame."""
    rows = []
    for s in shots:
        rows.append({
            "match_id": s.get("match_id"),
            "minute": int(s.get("minute")),
            "team": s.get("h_team") if s.get("h_a") == "h" else s.get("a_team"),
            "h_a": s.get("h_a"),
            "player": s.get("player"),
            "result": s.get("result"),
            "X": float(s.get("X")),
            "Y": float(s.get("Y")),
            "is_goal": 1 if s.get("result") == "Goal" else 0,
            "situation": s.get("situation")
        })
    df = pd.DataFrame(rows)
    return df.dropna(subset=["X", "Y"]).reset_index(drop=True)

In [ ]:
# Descargar IDs de partidos
match_ids, raw_matches = get_league_match_ids(LEAGUE, SEASON)
print(f"Partidos encontrados: {len(match_ids)}")

# Descargar tiros
all_shots = []
with UnderstatClient() as us:
    for mid in tqdm(match_ids, desc=f"Descargando tiros {LEAGUE}"):
        try:
            all_shots.extend(fetch_match_shots(mid, us))
        except Exception:
            pass
        sleep(0.15)

print(f"Tiros descargados: {len(all_shots)}")

---
## 2. Preprocesamiento

- Convertir a DataFrame
- Filtrar penales y tiros libres directos (solo open play)
- Estandarizar coordenadas

In [ ]:
# Convertir a DataFrame
df = shots_to_df(all_shots)
print(f"Tiros totales: {len(df)}")

# Filtrar penales y tiros libres directos
df = df[~df["situation"].isin(["Penalty", "DirectFreekick"])]
print(f"Tiros open play: {len(df)}")

# Estandarizar coordenadas (todos los tiros hacia X=1)
mask = df["X"] < 0.5
df.loc[mask, "X"] = 1 - df.loc[mask, "X"]

df.sample(5)

---
## 3. Distancia Euclidiana

Calculamos la distancia **total** desde el punto del tiro hasta el centro de la portería usando el teorema de Pitágoras:

$$d_{euclid} = \sqrt{d_x^2 + d_y^2}$$

Donde:
- $d_x$ = distancia longitudinal a la línea de gol (en metros)
- $d_y$ = distancia lateral al centro de la portería (en metros)

Esto es más preciso que usar solo la distancia longitudinal, ya que captura que un tiro desde el costado tiene menor probabilidad de gol.

In [ ]:
# Calcular distancias
df["dist_long_m"] = (1.0 - df["X"]) * PITCH_LENGTH_M
df["dist_lateral_m"] = np.abs(df["Y"] - GOAL_CENTER_Y) * PITCH_WIDTH_M
df["dist_euclid_m"] = np.sqrt(df["dist_long_m"]**2 + df["dist_lateral_m"]**2)

print("Comparación de distancias (muestra):")
df[["X", "Y", "dist_long_m", "dist_lateral_m", "dist_euclid_m"]].sample(5).round(2)

In [ ]:
# Visualizar distribución de distancias
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df["dist_euclid_m"], bins=40, edgecolor="black", alpha=0.7)
axes[0].set_xlabel("Distancia euclidiana (m)")
axes[0].set_ylabel("Frecuencia")
axes[0].set_title("Distribución de distancias de tiro")
axes[0].axvline(D_MIN, color="red", ls="--", label=f"D_MIN={D_MIN}m")
axes[0].axvline(D_MAX, color="red", ls="--", label=f"D_MAX={D_MAX}m")
axes[0].legend()

# Scatter plot de posiciones
axes[1].scatter(df["X"] * 100, df["Y"] * 65, c=df["is_goal"], cmap="RdYlGn", alpha=0.5, s=10)
axes[1].set_xlabel("X (m)")
axes[1].set_ylabel("Y (m)")
axes[1].set_title("Posición de tiros (verde=gol)")
axes[1].axvline(100, color="black", lw=2)

plt.tight_layout()
plt.show()

---
## 4. Modelo de Decaimiento Exponencial

Usamos un modelo de **decaimiento exponencial** para estimar la probabilidad de gol:

$$xG = e^{-d/k} \times a + b$$

Donde:
- $d$ = distancia euclidiana al centro de la portería
- $k$ = constante de decaimiento (controla la rapidez del decaimiento)
- $a$ = multiplicador
- $b$ = intercepto (probabilidad base a distancias largas)

**Ventaja**: A diferencia de una curva de potencia, este modelo garantiza que $xG < 1$ para cualquier distancia.

In [ ]:
# Crear bins de distancia
bins = np.arange(0.0, np.ceil(df["dist_euclid_m"].max()) + BIN_WIDTH, BIN_WIDTH)
df["dist_bin"] = pd.cut(df["dist_euclid_m"], bins=bins, right=False)

# Calcular estadísticas por bin
bin_stats = (
    df.groupby("dist_bin", observed=True)
    .agg(
        shots=("is_goal", "size"),
        goals=("is_goal", "sum"),
        d_min=("dist_euclid_m", "min"),
        d_max=("dist_euclid_m", "max")
    )
    .dropna(subset=["d_min", "d_max"])
    .reset_index(drop=True)
)

bin_stats["d_mid"] = 0.5 * (bin_stats["d_min"] + bin_stats["d_max"])

# Suavizado de Laplace para probabilidad
bin_stats["p_goal"] = (bin_stats["goals"] + ALPHA) / (bin_stats["shots"] + 2 * ALPHA)

print(f"Bins creados: {len(bin_stats)}")
bin_stats.head(10)

In [ ]:
# Filtrar bins para regresión (evitar ruido en colas)
fit_bins = bin_stats[
    (bin_stats["shots"] >= LOW_N) &
    (bin_stats["d_mid"] >= D_MIN) &
    (bin_stats["d_mid"] <= D_MAX)
].copy()

print(f"Bins para regresión: {len(fit_bins)} (de {len(bin_stats)} totales)")
print(f"Rango de distancia: {fit_bins['d_mid'].min():.1f}m - {fit_bins['d_mid'].max():.1f}m")

In [ ]:
# Definir modelo exponencial
def exp_decay_xg(d, k, a, b):
    """Modelo de decaimiento exponencial: xG = e^(-d/k) * a + b"""
    return np.exp(-d / k) * a + b

# Datos para ajuste
d_data = fit_bins["d_mid"].values
p_data = fit_bins["p_goal"].values
w_data = fit_bins["shots"].values  # Pesos por número de tiros

# Ajustar modelo
p0 = [5.0, 0.9, 0.03]  # Valores iniciales: k, a, b
popt, pcov = curve_fit(exp_decay_xg, d_data, p_data, p0=p0, sigma=1/np.sqrt(w_data), maxfev=5000)
k_opt, a_opt, b_opt = popt

# Calcular R²
p_pred = exp_decay_xg(d_data, k_opt, a_opt, b_opt)
r2 = r2_score(p_data, p_pred)

print("=" * 50)
print("MODELO AJUSTADO")
print("=" * 50)
print(f"\nEcuación: xG = e^(-d/{k_opt:.2f}) × {a_opt:.4f} + {b_opt:.4f}")
print(f"\nParámetros:")
print(f"  k = {k_opt:.4f} (constante de decaimiento)")
print(f"  a = {a_opt:.4f} (multiplicador)")
print(f"  b = {b_opt:.4f} (intercepto)")
print(f"\nR² = {r2:.4f}")

# Verificar comportamiento en d=0
xg_at_zero = exp_decay_xg(0, k_opt, a_opt, b_opt)
print(f"\nVerificación: xG(d=0) = {xg_at_zero:.4f} (< 1.0 ✓)")

In [ ]:
# Visualizar modelo ajustado
d_range = np.linspace(0.5, D_MAX + 5, 300)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel izquierdo: Todos los bins vs modelo
ax1 = axes[0]
ax1.scatter(bin_stats["d_mid"], bin_stats["p_goal"], s=20, alpha=0.4, label="Todos los bins")
ax1.scatter(fit_bins["d_mid"], fit_bins["p_goal"], s=40, alpha=0.9, label="Bins para ajuste")
ax1.plot(d_range, exp_decay_xg(d_range, k_opt, a_opt, b_opt), lw=2, color="purple",
         label=f"Modelo: R²={r2:.3f}")
ax1.axvspan(D_MIN, D_MAX, color="grey", alpha=0.1)
ax1.set_xlabel("Distancia euclidiana (m)")
ax1.set_ylabel("Probabilidad de gol")
ax1.set_title("Ajuste del modelo exponencial")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Panel derecho: Curva completa con límite
ax2 = axes[1]
d_full = np.linspace(0, 40, 300)
ax2.plot(d_full, exp_decay_xg(d_full, k_opt, a_opt, b_opt), lw=2, color="purple")
ax2.axhline(1.0, ls="--", color="red", alpha=0.5, label="Límite p=1.0")
ax2.scatter([0], [xg_at_zero], s=100, color="orange", marker="*", zorder=5,
            label=f"d=0: {xg_at_zero:.2f}")
ax2.set_xlabel("Distancia euclidiana (m)")
ax2.set_ylabel("xG")
ax2.set_title(f"xG = e^(-d/{k_opt:.1f}) × {a_opt:.2f} + {b_opt:.2f}")
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

In [ ]:
# Aplicar modelo a todos los tiros
df["xg"] = np.clip(exp_decay_xg(df["dist_euclid_m"], k_opt, a_opt, b_opt), 1e-9, 1 - 1e-9)

# Crear columna venue (local/visitante)
df["venue"] = df["h_a"].map({"h": "home", "a": "away"})

print(f"xG calculado para {len(df)} tiros")
df[["player", "team", "dist_euclid_m", "xg", "is_goal"]].sample(5).round(3)

---
## 5. Análisis por Equipo

Aplicamos el modelo para analizar el rendimiento ofensivo y defensivo de cada equipo.

In [ ]:
# Construir matriz de partidos (xG por equipo local y visitante)
def build_match_matrix(df):
    agg = df.groupby(["match_id", "team", "venue"], as_index=False).agg(xg=("xg", "sum"))
    home = agg[agg["venue"] == "home"][["match_id", "team", "xg"]].rename(
        columns={"team": "home_team", "xg": "home_xg"}
    )
    away = agg[agg["venue"] == "away"][["match_id", "team", "xg"]].rename(
        columns={"team": "away_team", "xg": "away_xg"}
    )
    return home.merge(away, on="match_id", how="inner")

match_matrix = build_match_matrix(df)
print(f"Partidos analizados: {len(match_matrix)}")

In [ ]:
# RANKING OFENSIVO: Equipos que generan más xG
xg_generado_local = match_matrix.groupby("home_team")["home_xg"].sum()
xg_generado_visita = match_matrix.groupby("away_team")["away_xg"].sum()

ranking_ofensivo = (xg_generado_local.add(xg_generado_visita, fill_value=0)
                    .sort_values(ascending=False)
                    .reset_index())
ranking_ofensivo.columns = ["equipo", "xg_generado"]
ranking_ofensivo["partidos"] = len(match_matrix) // 10  # aprox partidos por equipo
ranking_ofensivo["xg_por_partido"] = ranking_ofensivo["xg_generado"] / ranking_ofensivo["partidos"]

print("RANKING OFENSIVO (equipos que generan más oportunidades)")
print("=" * 55)
ranking_ofensivo.round(2)

In [ ]:
# RANKING DEFENSIVO: Equipos que reciben más xG (problemas defensivos)
xg_recibido_local = match_matrix.groupby("home_team")["away_xg"].sum()  # xG del rival cuando juegas de local
xg_recibido_visita = match_matrix.groupby("away_team")["home_xg"].sum()  # xG del rival cuando juegas de visita

ranking_defensivo = (xg_recibido_local.add(xg_recibido_visita, fill_value=0)
                     .sort_values(ascending=False)
                     .reset_index())
ranking_defensivo.columns = ["equipo", "xg_recibido"]
ranking_defensivo["partidos"] = len(match_matrix) // 10
ranking_defensivo["xg_recibido_por_partido"] = ranking_defensivo["xg_recibido"] / ranking_defensivo["partidos"]

print("RANKING DEFENSIVO (equipos con más problemas defensivos)")
print("=" * 55)
ranking_defensivo.round(2)

In [ ]:
# Ver xG de un partido específico
def show_match_xg(match_id: str):
    """Muestra el xG de un partido específico."""
    match = match_matrix[match_matrix["match_id"] == match_id]
    if match.empty:
        print(f"Partido {match_id} no encontrado")
        return None
    
    row = match.iloc[0]
    print(f"\nPartido: {match_id}")
    print("=" * 40)
    print(f"{row['home_team']:>20} (L)  {row['home_xg']:.2f} xG")
    print(f"{row['away_team']:>20} (V)  {row['away_xg']:.2f} xG")
    return match

# Ejemplo: mostrar un partido aleatorio
sample_match_id = match_matrix["match_id"].sample(1).values[0]
show_match_xg(sample_match_id)

In [ ]:
# Buscar partidos de un equipo específico
EQUIPO = "Liverpool"

partidos_equipo = match_matrix[
    (match_matrix["home_team"] == EQUIPO) | (match_matrix["away_team"] == EQUIPO)
].copy()

print(f"Partidos de {EQUIPO}: {len(partidos_equipo)}\n")
partidos_equipo.round(2)